<a href="https://colab.research.google.com/github/GHROTH-L/crawler/blob/main/%E7%88%AC%E5%8F%96%E5%8D%9A%E5%AE%A2%E4%BE%86%E5%9C%96%E6%9B%B8%E8%B3%87%E8%A8%8A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import time

In [18]:
#產生頁數與關鍵字結果
def gen_url(url,start_page,end_page,key):
  urls= []
  for page in range(start_page, end_page+1): #range 是最後-1 所以這裡要加入
    urls.append(url.format(page,key))
  return urls

In [71]:
#價錢有的有標示打折 有的則是直接優惠價，會造成選取第一個或是第二個的問題
def get_second_b(b):
    if len(b) >= 2:
        return b[1].text.strip()
    else:
        return b[0].text.strip()

In [117]:
def get_books(soup):
  mods = soup.find_all("div",class_="mod2 table-container")
  books = []
  for mod in mods:
    tables = mod.find_all("div", class_="table-td")
    for table in tables:
      boxs = table.find_all("div", class_="box")
      authors =table.find_all("p", class_="author")
      prices = table.find_all("ul", class_="price clearfix")
      for box in boxs:
        title = box.a["title"]
        photo = box.a.img["data-src"]
        book_url= "http:"+box.a["href"]
      for a in authors:
        author = a.text.strip().replace('\xa0\xa0', ',').strip() #雙作者會有這個符號，所以取代掉
      for p in prices:
        #price = p.li.text.strip()
        price = get_second_b(p.li.select("b"))
      book = [title,photo,author,price,book_url]
      books.append(book)
  return books 

In [108]:
#爬蟲
def scraping(urls):
  booklist= [["書名","圖片","作者","價錢","網址"]]
  page = 1
  for url in urls:
    print("抓取：第"+str(page)+"頁的"+str(key)+"網路資料中...")
    headers = {"user-agen" :"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"}
    r = requests.get(url, headers=headers)
    page = page+1
    if r.status_code == requests.codes.ok:
      soup = BeautifulSoup(r.text,"html.parser")
      books = get_books(soup)
      booklist = booklist + books #在巢狀清單用加法
      time.sleep(5)
    else:
      print("請求錯誤")
  return booklist

  




In [127]:
if __name__ == "__main__":
  url ="https://search.books.com.tw/search/query/cat/all/sort/1/v/1/spell/3/ms2/ms2_1/page/{0}/key/{1}"
  key = str(input("請輸入要查找的項目："))
  start = int(input("請輸入開始頁碼："))
  end = int(input("請輸入結束頁碼："))
  urls= gen_url(url,start,end,key)
  booklist = scraping(urls)
  print("以下是查找关键字为" + str(key) + "的清单，为博客來第" + str(start) + "頁至" + str(end) + "頁")
  print(booklist)


請輸入要查找的項目：R語言
請輸入開始頁碼：1
請輸入結束頁碼：3
抓取：第1頁的R語言網路資料中...
抓取：第2頁的R語言網路資料中...
抓取：第3頁的R語言網路資料中...
以下是查找关键字为R語言的清单，为博客來第1頁至3頁
[['書名', '圖片', '作者', '價錢', '網址'], ['精通大數據!R 語言資料分析與應用 第二版', 'https://im1.book.com.tw/image/getImage?i=https://www.books.com.tw/img/001/078/27/0010782742.jpg&w=187&h=187&v=5ab0e2bb', 'Jared P. Lander,鍾振蔚', '684', 'http://search.books.com.tw/redirect/move/key/R%E8%AA%9E%E8%A8%80/area/mid/item/0010782742/page/1/idx/1/cat/001/pdf/1/spell/3'], ['R語言邁向Big Data之路：王者歸來(第二版)', 'https://im1.book.com.tw/image/getImage?i=https://www.books.com.tw/img/001/087/70/0010877047.jpg&w=187&h=187&v=5fb6492e', '洪錦魁,蔡桂宏', '616', 'http://search.books.com.tw/redirect/move/key/R%E8%AA%9E%E8%A8%80/area/mid/item/0010877047/page/1/idx/2/cat/001/pdf/1/spell/3'], ['R 語言資料分析活用範例詳解', 'https://im1.book.com.tw/image/getImage?i=https://www.books.com.tw/img/001/069/54/0010695444.jpg&w=187&h=187&v=563346f5', '姜葉飛,方匡南,朱建平,H&C', '468', 'http://search.books.com.tw/redirect/move/key/R%E8%AA%9E%E8%A8%80/area/mid/i